In [7]:
import os
import sys
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import ast
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
sys.path.insert(0, os.path.abspath("../../" + 'src/scraping'))
import metadata as meta
sys.path.insert(0, os.path.abspath("../../" + "src/modeling"))
import basic_stats as basic
import time
from PIL import Image
import datetime

In [ ]:
def get_all_thumbnails(save_dir, chan_ids, master_df):
    for chan_id in chan_ids:
        chan_df = master_df[master_df['channelId'] == chan_id]
        chan_vids = meta.download_df_thumbs(chan_df,save_dir,'medium')
    chan_basic_df = basic.basic_image_stats(save_dir)
    full_chan = master_df.merge(chan_basic_df,on='videoId')
    full_chan = full_chan.drop_duplicates(subset='videoId')
    return full_chan

def get_corr(img_feats, meta_feats,df,sign):
    corrs = []
    for col in meta_feats:
        for col2 in img_feats:
        #     col = 'z_views'
        #     col2 = 'contrast'
            step1 = df[df[col].notnull()]
            filtered = step1[step1[col2].notnull()]
            corr = np.corrcoef(filtered[col],filtered[col2])[0][1]
            corrs.append([col2, col, corr])
#             if sign == "-":
#                 if corr < 0:
#                     print(col, col2, corr)
#                     print('---------------')
#             else:
#                 if corr > 0:
#                     print(col, col2, corr)
#                     print('---------------')
    corr_df = pd.DataFrame(corrs, columns=['img_ft','meta_ft','corr'])
    return corr_df


# plots single emoition vs z views for all specified emotions
def get_emotions_df(emotions,in_df,meta_col):
    emotions_data = []
    for targ in emotions:
        emotion = in_df['emotions'].apply(lambda x: targ in x if type(x) == str else x)
        emotions_data.append(in_df[emotion == True][meta_col].describe())
    #     plt.scatter(emotion,face_data['z_views'])
    #     plt.title(targ + " vs. z_views")
    #     plt.xlabel(targ)
    #     plt.ylabel('z_views')
    #     plt.show()
    emotions_df = pd.DataFrame(emotions_data).reset_index()
    emotions_df['emotion'] = emotions
    return emotions_df
                    
def get_range(col_name, df):
    stats = df[col_name].describe()
    col_range = np.arange(stats['25%'],stats['max'],stats['std']/3)
    return col_range

def tune_params(df, c1, c2, meta_col):
    baseline_stats = df[meta_col].describe()
    base_mean = baseline_stats['mean']
    base_median = baseline_stats['50%']
#     print("baseline:     ",
#           "count:",baseline_stats['count'],
#           "mean:",baseline_stats['mean'].round(3),
#           "median:",baseline_stats['50%'].round(3))
    rel_stats = []
    c1_range = get_range(c1,df)
    c2_range = get_range(c2,df)
    for c1_cutoff in c1_range:
        for c2_cutoff in c2_range:
            filtered = df.apply(lambda x: x[c1] > c1_cutoff and x[c2] > c2_cutoff,axis=1)
            stats = df[filtered][meta_col].describe()
            cur_count = stats['count']
            cur_mean = stats['mean']
            cur_median = stats['50%']
            cur_stats_list = [c1,c1_cutoff,c2,c2_cutoff,cur_count, cur_mean, cur_median]
            rel_stats.append(cur_stats_list)
    res_df = pd.DataFrame(rel_stats, columns=['col1','col1cutoff','col2','col2cutoff','count','mean','median'])
    relevant_df = res_df[res_df['count'] > 0] # gets rid of param combinations with no results
    # filters out param combinations that did worse than the baseline
    better_df = relevant_df[relevant_df.apply(
        lambda x: x['mean'] > base_mean and x['median'] > base_median,axis=1)]
    return better_df

In [ ]:
# config variables
emotions = ['angry', 'happy', 'sad', 'fear', 'neutral', 'surprise']
describe_cols = ['mean','std','25%','50%','75%']
numerical_img_feats = ['unique_rgb_ratio','mean_hue',
                       'mean_saturation','mean_brightness', 'contrast',
                       'edge_score','numFaces']
meta_feats = ['viewCount','z_views']
master_df = pd.read_csv("../../data/local/fortnite/video_data/fortnite_master_metadata_updated_facial_features.csv")
save_dir = "../../data/local/fortnite/thumbnails/"
chan_ids = all_meta['channelId'].value_counts().index

In [ ]:
# downloads thumbnails not in local storage and computes basic image statistics, takes a decent bit
final_df = get_all_thumbnails(save_dir, chan_ids, master_df)

In [ ]:
# different subsets of data
final_df = final_df[final_df['z_views'].notnull()]
face_data = final_df[final_df['numFaces'].notnull()]
no_face_data = final_df[final_df['numFaces'].isnull()]
all_face = final_df.fillna(value={"numFaces":0})
# dataset to use for eda
analysis_df = all_face

In [ ]:
# emotion stats for each descriptive stat for each meta column
plt.rcParams['figure.figsize'] = [6, 4]
emotions_df = get_emotions_df(emotions,analysis_df,meta_feats[0])
plt.bar(emotions_df['emotion'],emotions_df['count'],color=['red','green','blue','purple','grey','orange'])
plt.xlabel('emotion')
plt.ylabel('count')
plt.title("Video count per emotion")
plt.show()
for meta_feat in meta_feats:
    emotions_df = get_emotions_df(emotions,analysis_df,meta_feat)
    for col in describe_cols:
        plt.bar(emotions_df['emotion'],emotions_df[col],color=['red','green','blue','purple','grey','orange'])
        plt.xlabel('emotion')
        plt.ylabel(col)
        plt.title(meta_feat + " (" + col + ") per emotion")
        plt.show()

In [ ]:
# table of corr coeffecients for convenience
all_corrs = get_corr(numerical_img_feats, meta_feats, analysis_df, '-')
all_corrs

In [ ]:
# plot summary of each numerical image feature vs each meta column with correlation in title
for col in numerical_img_feats:
    for meta_col in meta_feats:
        plt.scatter(analysis_df[col],analysis_df[meta_col])
        plt.xlabel(col)
        plt.ylabel(meta_col)
        cur_corr_df = all_corrs[all_corrs.apply(lambda x: x['img_ft'] == col and x['meta_ft'] == meta_col,axis=1)]
        cur_corr = cur_corr_df.iloc[0]['corr']
        plt.title(col + " vs. " + meta_col + " Corr: " + str(cur_corr.round(3)))
        plt.show()

In [ ]:
summary_df = pd.DataFrame()
for meta_feat in meta_feats:
    cur_corr_df = all_corrs[all_corrs['meta_ft'] == meta_feat]
    c1, c2 = cur_corr_df.sort_values(by='corr',ascending=False)['img_ft'].iloc[0:2].values
    # this takes a while bc of the many combinations, may re-write the way I get the ranges
    tuned_df = tune_params(analysis_df,c1,c2,meta_feat) 
    tuned_df['mm_sum'] = tuned_df['mean'] + tuned_df['median']
    #old cell break
    top_df = tuned_df.sort_values(by='mm_sum',ascending=False)
    rgb_cutoff = top_df['col1cutoff'].describe()['mean']
    sat_cutoff = top_df['col2cutoff'].describe()['mean']
    top_cut = analysis_df[analysis_df.apply(
        lambda x: x[c1] > rgb_cutoff and x[c2] > sat_cutoff,axis=1)]

    baseline_stats = analysis_df[meta_feat].describe().round(4)
    baseline_stats['type'] = 'baseline'
    baseline_stats['meta_col'] = baseline_stats.name

    top_cut_stats = top_cut[meta_feat].describe().round(4)
    top_cut_stats['type'] = 'top_cut'
    top_cut_stats['meta_col'] = top_cut_stats.name

    cur_summary_df = pd.DataFrame([baseline_stats,top_cut_stats]).reset_index(drop=True)
    summary_df = pd.concat([summary_df,cur_summary_df])

In [ ]:
# analysis using threshold cutoff for 2 features with the highest + correlation to the meta_col
# results are stats of the "top cut" that meet said criteria
summary_df

In [ ]:
# not adjusted for multiple meta feats
targ_col = 'z_views'
X_train, X_test, y_train, y_test = train_test_split(analysis_df[numerical_img_feats],
                                                    analysis_df[targ_col],
                                                    train_size=.9)
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
print("RF Score:",rf.score(X_test,y_test))
plt.scatter(rf_preds,y_test)
plt.title('RF ' + targ_col + ' Predictions vs Actual')
plt.xlabel('prediction')
plt.ylabel('actual')
plt.show()
# feat_importances = pd.Series(dict(zip(numerical_img_feats,rf.feature_importances_)))
# print("RF Feat Importances:\n",feat_importances.sort_values(ascending=False))
gbr = GradientBoostingRegressor()
gbr.fit(X_train,y_train)
gbr_preds = gbr.predict(X_test)
print("GBR Score:",gbr.score(X_test,y_test))
plt.scatter(gbr_preds,y_test)
plt.title('GBR ' + targ_col + ' Predictions vs Actual')
plt.xlabel('prediction')
plt.ylabel('actual')
plt.show()

In [ ]:
high_lvl_df = pd.DataFrame()
for num_feat in numerical_img_feats:
    for num_feat2 in numerical_img_feats:
        for num_feat3 in numerical_img_feats:
            for num_feat4 in numerical_img_feats:
                col_name = num_feat + "/" + num_feat2 + "/" + num_feat3 + "/" + num_feat4 
                high_lvl_df[col_name] = analysis_df[num_feat]*analysis_df[num_feat2]*analysis_df[num_feat3]*analysis_df[num_feat4]
high_lvl_df

In [ ]:
high_train, high_test, targ_train, targ_test = train_test_split(high_lvl_df, analysis_df['viewCount'])
lr = LinearRegression()
lr.fit(high_train, targ_train)
lr.score(high_test, targ_test)

In [ ]:
lr_preds = lr.predict(high_test)

In [ ]:
plt.scatter(lr_preds, targ_test)

In [ ]:
analysis_df['has_face'] = analysis_df['numFaces'] > 0
poor_perf = analysis_df[analysis_df['z_views'] < -1]
good_perf = analysis_df[analysis_df['z_views'] > 1]

In [ ]:
poor_data = []
good_data = []
for num_feat in numerical_img_feats:
    poor_data.append(poor_perf[num_feat].describe())
    good_data.append(good_perf[num_feat].describe())
poor_stats = pd.DataFrame(poor_data).drop("count",axis=1).T
good_stats = pd.DataFrame(good_data).drop('count',axis=1).T

In [ ]:
plt.rcParams['figure.figsize'] = [12, 5]
x = np.arange(len(good_stats.index))  # the label locations
width = 0.35  # the width of the bars
for num_feat in numerical_img_feats:
    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2,good_stats[num_feat],width,label='Good Performance')
    rects2 = ax.bar(x + width/2,poor_stats[num_feat],width,label='Poor Performance')
    ax.set_xticks(x)
    ax.set_xticklabels(good_stats.index)
    ax.set_title(num_feat)
    ax.legend()
    plt.show()

In [3]:
new_df = pd.read_csv("../../data/local/fortnite/video_data/full_features.csv")

In [5]:
new_df.columns

Index(['Unnamed: 0', 'categoryId', 'channelId', 'channelTitle', 'commentCount',
       'date', 'defaultLanguage', 'description', 'dislikeCount', 'duration',
       'favoriteCount', 'likeCount', 'tags', 'thumbnails', 'title', 'videoId',
       'viewCount', 'z_comments', 'z_dislikes', 'z_likes', 'z_views',
       'position', 'thumbnailFilename', 'size', 'num_rgb', 'unique_rgb_ratio',
       'mean_hue', 'mean_saturation', 'mean_brightness', 'contrast',
       'edge_score', 'numFaces', 'emotions', 'age', 'gender', 'race',
       'face_locations'],
      dtype='object')

# Old

In [8]:
dev = {"devices" : [{'status': 'ERROR', 'swver': None, 'ip': '192.168.1.106', 'devID': 5890359, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': 32.856, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': -117.212, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890874, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': 34.2192, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': -118.352, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890880, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890881, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890882, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890883, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890884, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890885, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890886, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890887, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890888, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890889, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890890, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890891, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890892, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890893, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890894, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890895, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890896, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890897, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890898, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890899, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890900, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890901, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 55), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890902, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890903, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890904, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890905, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 16, 59), 'rssi': None, 'dev_long': None, 'groupID': 5}]}

In [11]:
pd.DataFrame(dev['devices'])

,status,swver,ip,devID,hw,mac,dev_lat,lastseen,rssi,dev_long,groupID
0,ERROR,None,192.168.1.106,5890359,None,CC:50:E3:A8:EB:3C,32.8560,2020-05-15 13:16:59,None,-117.212,5
1,None,None,192.168.1.30,5890874,None,CC:50:E3:B0:21:8C,34.2192,2020-05-15 13:16:55,None,-118.352,5
2,None,None,192.168.1.106,5890880,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:16:59,None,NaN,5
3,None,None,192.168.1.30,5890881,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:16:55,None,NaN,5
4,None,None,192.168.1.106,5890882,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:16:59,None,NaN,5
5,None,None,192.168.1.106,5890883,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:16:59,None,NaN,5
6,None,None,192.168.1.30,5890884,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:16:55,None,NaN,5
7,None,None,192.168.1.106,5890885,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:16:59,None,NaN,5
8,None,None,192.168.1.30,5890886,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:16:55,None,NaN,5
9,None,None,192.168.1.30,5890887,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:16:55,None,NaN,5


In [12]:
dev2 = {"devices" : [{'status': 'ERROR', 'swver': None, 'ip': '192.168.1.106', 'devID': 5890359, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': 32.856, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': -117.212, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890874, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': 34.2192, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': -118.352, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890880, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890881, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890882, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890883, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890884, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890885, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890886, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890887, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890888, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890889, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890890, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890891, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890892, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890893, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890894, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890895, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890896, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890897, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890898, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890899, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890900, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890901, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 45), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890902, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890903, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890904, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.106', 'devID': 5890905, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 13, 41, 49), 'rssi': None, 'dev_long': None, 'groupID': 5}]}

In [13]:
pd.DataFrame(dev2['devices'])

,status,swver,ip,devID,hw,mac,dev_lat,lastseen,rssi,dev_long,groupID
0,ERROR,None,192.168.1.106,5890359,None,CC:50:E3:A8:EB:3C,32.8560,2020-05-15 13:41:49,None,-117.212,5
1,None,None,192.168.1.30,5890874,None,CC:50:E3:B0:21:8C,34.2192,2020-05-15 13:41:45,None,-118.352,5
2,None,None,192.168.1.106,5890880,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:41:49,None,NaN,5
3,None,None,192.168.1.30,5890881,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:41:45,None,NaN,5
4,None,None,192.168.1.106,5890882,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:41:49,None,NaN,5
5,None,None,192.168.1.106,5890883,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:41:49,None,NaN,5
6,None,None,192.168.1.30,5890884,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:41:45,None,NaN,5
7,None,None,192.168.1.106,5890885,None,CC:50:E3:A8:EB:3C,NaN,2020-05-15 13:41:49,None,NaN,5
8,None,None,192.168.1.30,5890886,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:41:45,None,NaN,5
9,None,None,192.168.1.30,5890887,None,CC:50:E3:B0:21:8C,NaN,2020-05-15 13:41:45,None,NaN,5


In [16]:
dev3 = {"devices" : [{'status': 'OK', 'swver': None, 'ip': '192.168.10.1', 'devID': 1, 'hw': None, 'mac': '80:7D:3A:A2:9C:08', 'dev_lat': 32.883, 'lastseen': datetime.datetime(2020, 5, 5, 22, 16, 12), 'rssi': -63.0, 'dev_long': -117.234, 'groupID': 0},{'status': 'OK', 'swver': None, 'ip': '192.168.1.175', 'devID': 2, 'hw': None, 'mac': '3C:71:BF:63:DC:D0', 'dev_lat': 32.8512, 'lastseen': datetime.datetime(2020, 5, 15, 15, 7), 'rssi': None, 'dev_long': -117.217, 'groupID': 22},{'status': 'ERROR', 'swver': None, 'ip': '192.168.1.46', 'devID': 1144, 'hw': None, 'mac': '3C:71:BF:63:EE:0C', 'dev_lat': 34.1542, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 56), 'rssi': None, 'dev_long': -118.045, 'groupID': 16},{'status': 'ERROR', 'swver': None, 'ip': '192.168.1.5', 'devID': 5890336, 'hw': None, 'mac': '3C:71:BF:6C:A9:7C', 'dev_lat': 32.8741, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 52), 'rssi': None, 'dev_long': -117.218, 'groupID': 10},{'status': 'ERROR', 'swver': None, 'ip': '192.168.1.106', 'devID': 5890359, 'hw': None, 'mac': 'CC:50:E3:A8:EB:3C', 'dev_lat': 32.856, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 59), 'rssi': None, 'dev_long': -117.212, 'groupID': 5},{'status': 'ERROR', 'swver': None, 'ip': '10.0.0.158', 'devID': 5890475, 'hw': None, 'mac': '3C:71:BF:6C:62:2C', 'dev_lat': 37.3969, 'lastseen': datetime.datetime(2020, 5, 5, 20, 39, 33), 'rssi': None, 'dev_long': -121.943, 'groupID': 2},{'status': 'OK', 'swver': None, 'ip': '192.168.1.51', 'devID': 5890868, 'hw': None, 'mac': '3C:71:BF:64:36:C0', 'dev_lat': 32.8472, 'lastseen': datetime.datetime(2020, 5, 10, 21, 44, 35), 'rssi': None, 'dev_long': -117.196, 'groupID': 2},{'status': 'OK', 'swver': None, 'ip': '192.168.0.17', 'devID': 5890870, 'hw': None, 'mac': '80:7D:3A:E9:67:5C', 'dev_lat': 32.869, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 50), 'rssi': None, 'dev_long': -117.218, 'groupID': 11},{'status': None, 'swver': None, 'ip': '192.168.0.14', 'devID': 5890871, 'hw': None, 'mac': '3C:71:BF:63:DF:98', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 56), 'rssi': None, 'dev_long': None, 'groupID': 21},{'status': None, 'swver': None, 'ip': None, 'devID': 5890872, 'hw': None, 'mac': '3C:71:BF:62:E7:FC', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 59), 'rssi': None, 'dev_long': None, 'groupID': 11},{'status': 'OK', 'swver': None, 'ip': '192.168.1.134', 'devID': 5890873, 'hw': None, 'mac': '80:7D:3A:BC:7F:00', 'dev_lat': 47.6131, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 51), 'rssi': None, 'dev_long': -122.205, 'groupID': 19},{'status': None, 'swver': None, 'ip': '192.168.1.30', 'devID': 5890874, 'hw': None, 'mac': 'CC:50:E3:B0:21:8C', 'dev_lat': 34.2192, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 54), 'rssi': None, 'dev_long': -118.352, 'groupID': 5},{'status': None, 'swver': None, 'ip': '192.168.1.195', 'devID': 5890875, 'hw': None, 'mac': 'CC:50:E3:AF:E1:B4', 'dev_lat': 32.9269, 'lastseen': datetime.datetime(2020, 5, 15, 15, 7), 'rssi': None, 'dev_long': -117.195, 'groupID': 1},{'status': None, 'swver': None, 'ip': '192.168.0.6', 'devID': 5890876, 'hw': None, 'mac': 'CC:50:E3:B0:92:B4', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 12, 4, 24), 'rssi': None, 'dev_long': None, 'groupID': 23},{'status': None, 'swver': None, 'ip': '192.168.1.25', 'devID': 5890879, 'hw': None, 'mac': '3C:71:BF:64:21:DC', 'dev_lat': None, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 53), 'rssi': None, 'dev_long': None, 'groupID': 25},{'status': None, 'swver': None, 'ip': '192.168.0.75', 'devID': 5890906, 'hw': None, 'mac': '3C:71:BF:63:DF:70', 'dev_lat': 37.4019, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 58), 'rssi': None, 'dev_long': -121.931, 'groupID': 9},{'status': None, 'swver': None, 'ip': '192.168.1.92', 'devID': 5890907, 'hw': None, 'mac': '3C:71:BF:62:E4:30', 'dev_lat': 33.6962, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 51), 'rssi': None, 'dev_long': -117.79, 'groupID': 25},{'status': 'OK', 'swver': None, 'ip': '192.168.1.25', 'devID': 5890908, 'hw': None, 'mac': 'CC:50:E3:B0:21:A4', 'dev_lat': 33.705, 'lastseen': datetime.datetime(2020, 5, 15, 15, 6, 52), 'rssi': None, 'dev_long': -117.738, 'groupID': 15}]}

In [17]:
pd.DataFrame(dev3['devices'])

,status,swver,ip,devID,hw,mac,dev_lat,lastseen,rssi,dev_long,groupID
0,OK,None,192.168.10.1,1,None,80:7D:3A:A2:9C:08,32.8830,2020-05-05 22:16:12,-63.0,-117.234,0
1,OK,None,192.168.1.175,2,None,3C:71:BF:63:DC:D0,32.8512,2020-05-15 15:07:00,NaN,-117.217,22
2,ERROR,None,192.168.1.46,1144,None,3C:71:BF:63:EE:0C,34.1542,2020-05-15 15:06:56,NaN,-118.045,16
3,ERROR,None,192.168.1.5,5890336,None,3C:71:BF:6C:A9:7C,32.8741,2020-05-15 15:06:52,NaN,-117.218,10
4,ERROR,None,192.168.1.106,5890359,None,CC:50:E3:A8:EB:3C,32.8560,2020-05-15 15:06:59,NaN,-117.212,5
5,ERROR,None,10.0.0.158,5890475,None,3C:71:BF:6C:62:2C,37.3969,2020-05-05 20:39:33,NaN,-121.943,2
6,OK,None,192.168.1.51,5890868,None,3C:71:BF:64:36:C0,32.8472,2020-05-10 21:44:35,NaN,-117.196,2
7,OK,None,192.168.0.17,5890870,None,80:7D:3A:E9:67:5C,32.8690,2020-05-15 15:06:50,NaN,-117.218,11
8,None,None,192.168.0.14,5890871,None,3C:71:BF:63:DF:98,NaN,2020-05-15 15:06:56,NaN,NaN,21
9,None,None,None,5890872,None,3C:71:BF:62:E7:FC,NaN,2020-05-15 15:06:59,NaN,NaN,11


In [19]:
dev4 = [{'timestamp': datetime.datetime(2020, 5, 15, 20, 15, 30), 'blelog_id': 103895808, 'gid': 5, 'devmac': 'CC:50:E3:A8:EB:3C', 'blerssi': -87.0, 'blemac': '43:a7:36:d1:35:a7'},{'timestamp': datetime.datetime(2020, 5, 15, 20, 15), 'blelog_id': 103895227, 'gid': 5, 'devmac': 'CC:50:E3:A8:EB:3C', 'blerssi': -86.0, 'blemac': '43:a7:36:d1:35:a7'},{'timestamp': datetime.datetime(2020, 5, 15, 20, 14, 1), 'blelog_id': 103894224, 'gid': 5, 'devmac': 'CC:50:E3:A8:EB:3C', 'blerssi': -88.0, 'blemac': '43:a7:36:d1:35:a7'},{'timestamp': datetime.datetime(2020, 5, 15, 20, 13, 30), 'blelog_id': 103893666, 'gid': 5, 'devmac': 'CC:50:E3:A8:EB:3C', 'blerssi': -88.0, 'blemac': '43:a7:36:d1:35:a7'},{'timestamp': datetime.datetime(2020, 5, 15, 20, 13, 20), 'blelog_id': 103893547, 'gid': 5, 'devmac': 'CC:50:E3:A8:EB:3C', 'blerssi': -87.0, 'blemac': '43:a7:36:d1:35:a7'}]
pd.DataFrame(dev4)

,timestamp,blelog_id,gid,devmac,blerssi,blemac
0,2020-05-15 20:15:30,103895808,5,CC:50:E3:A8:EB:3C,-87.0,43:a7:36:d1:35:a7
1,2020-05-15 20:15:00,103895227,5,CC:50:E3:A8:EB:3C,-86.0,43:a7:36:d1:35:a7
2,2020-05-15 20:14:01,103894224,5,CC:50:E3:A8:EB:3C,-88.0,43:a7:36:d1:35:a7
3,2020-05-15 20:13:30,103893666,5,CC:50:E3:A8:EB:3C,-88.0,43:a7:36:d1:35:a7
4,2020-05-15 20:13:20,103893547,5,CC:50:E3:A8:EB:3C,-87.0,43:a7:36:d1:35:a7


In [ ]:
# all_meta = pd.read_csv("../../data/local/fortnite/video_data/fortnite_master_metadata.csv")

In [ ]:
# big_df = pd.read_csv('../../data/local/fortnite/video_data/merged_df.csv').drop("Unnamed: 0",axis=1)

In [ ]:
# no_faces = final_df[final_df['numFaces'].isnull()]['videoId']
# faces = final_df[final_df['numFaces'].notnull()]['videoId']

In [ ]:
# for i in np.random.choice(len(faces),size=20):
#     display(Image.open("../../data/local/fortnite/thumbnails/" + faces.iloc[i] + ".jpg"))
#     print(final_df.iloc[i]['numFaces'])

In [ ]:
# for i in np.random.choice(len(no_faces),size=20):
#     display(Image.open("../../data/local/fortnite/thumbnails/" + no_faces.iloc[i] + ".jpg"))

In [ ]:
# emotion = sorted_df['emotions'].apply(lambda x: len(ast.literal_eval(x)) if type(x) == str else 0)
# # print("Emotion:",targ)
# display(emotion.describe())
# plt.scatter(emotion,sorted_df['z_views'])
# # plt.show()

In [ ]:
# numerical_img_feats = ['unique_rgb_ratio','num_rgb','mean_hue',
#                        'mean_saturation','mean_brightness', 'contrast',
#                        'edge_score','numFaces']
# meta_feats = ['z_views']

In [ ]:
# get_corr(numerical_img_feats, meta_feats, face_data, '-')

In [ ]:
# get_corr(numerical_img_feats, meta_feats, face_data, '+')

In [ ]:
# c1 = 'num_rgb'
# c2 = 'edge_score'

In [ ]:
# tuned_df = tune_params(face_data,c1,c2)
# tuned_df['mm_sum'] = tuned_df['mean'] + tuned_df['median']

In [ ]:
# top_df = tuned_df.sort_values(by='mm_sum',ascending=False)
# rgb_cutoff = top_df['col1cutoff'].describe()['mean']
# sat_cutoff = top_df['col2cutoff'].describe()['mean']
# # face_data['z_views'].describe()
# top_cut = face_data[face_data.apply(
#     lambda x: x[c1] > rgb_cutoff and x[c2] > sat_cutoff,axis=1)]
# top_cut['z_views'].describe()

In [ ]:
# top_200 = pd.read_csv('../../data/local/fortnite/video_data/merged_df.csv').drop("Unnamed: 0",axis=1)
# full_views = top_200[top_200['z_views'].notnull()]

In [ ]:
# rf = RandomForestRegressor()
# rf.fit(full_views[basic_img_feats],full_views[['position']])
# rf.feature_importances_

In [ ]:
# baseline_stats = sorted_df['z_views'].describe()
# print("baseline:     ",
#       "count:",baseline_stats['count'],
#       "mean:",baseline_stats['mean'].round(3),
#       "median:",baseline_stats['50%'].round(3))
# # print('---------------------------------------')
# #['baseline',0,baseline_stats['count'],baseline_stats['mean'],baseline_stats['50%']]
# rel_stats = []
# c1 = 'num_rgb'
# c1_range = get_range(c1,sorted_df)
# c2 = 'mean_saturation'
# c2_range = get_range(c2, sorted_df)
# for c1_cutoff in c1_range:
#     for c2_cutoff in c2_range:
#         filtered = sorted_df.apply(lambda x: x[c1] > c1_cutoff and x[c2] > c2_cutoff,axis=1)
#         stats = sorted_df[filtered]['z_views'].describe()
#         cur_count = stats['count']
#         cur_mean = stats['mean']
#         cur_median = stats['50%']
#         cur_stats_list = [c1,c1_cutoff,c2,c2_cutoff,cur_count, cur_mean, cur_median]
#         rel_stats.append(cur_stats_list)
# res_df = pd.DataFrame(rel_stats, columns=['col1','col1cutoff','col2','col2cutoff','count','mean','median'])

In [ ]:
# not_null_df = res_df[res_df['mean'].notnull()]
# not_null_df[not_null_df.apply(lambda x: x['mean'] > .5 and x['median']> .2,axis=1)]

In [ ]:
# c1 = 'mean_saturation'
# c1_stats = sorted_df[c1].describe()
# c1_range = np.arange(c1_stats['25%'],c1_stats['75%'],c1_stats['std']/3)
# c1_range

In [ ]:
# col = 'contrast'
# print(sorted_df[col].describe())
# plt.scatter(sorted_df['contrast'],sorted_df['z_views'])

In [ ]:
# for vid_id in basic_stats_df['videoId'].values:
#     if vid_id not in df['videoId'].values:
#         print(vid_id)

In [ ]:
# merged_df = df.merge(basic_stats_df,how="left",on="videoId")

In [ ]:
# datetime.now().strftime("_%m_%d_%y")